In [1]:
from lark import Lark, Transformer, v_args

In [2]:
kumir_robot = """
    start: "использовать" "Робот"i alg+

    alg: "алг" name body 

    name: CHAR | CHAR name
    
    body: "нач" command+ "кон"

    command: cycle | (action";")* action | if_expression 

    action: /(вперед| вперёд)/i | /назад/i | /влево/i | /вправо/i | /закрасить/i

    cycle: "нц" (NUM /раз/ | /пока/ condition | for_cycle) command+ ("кц" | "кц_при" condition)

    for_cycle: /для/ VAR /от/ NUM /до/ NUM

    condition: NOT? direction wall

    wall: /свободно/ | /стена/

    direction: /слева/i | /справа/i | /сверху/i | /снизу/i

    if_expression: "если" condition "то" command+ "все" 

    NOT: /не/

    CHAR: /[|а-яА-Я0-9" "]+/

    VAR: /[a-zA-Z]/

    NUM: /[0-9]+/

%import common.NEWLINE
%import common.WS_INLINE
%ignore WS_INLINE
%ignore NEWLINE
"""

In [3]:
parser = Lark(kumir_robot, parser='lalr', transformer=None)

In [4]:
class KumirTransformer(Transformer):
    
    def start(self, items):
        return f'Используем Робот с алгоритмом:\n{items[1]}'

    def alg(self, items):
        return f'Алгоритм {items[0]}:\n{items[1]}'

    def body(self, items):
        return '\n'.join(items)

    def command(self, items):
        return '\n'.join(items)

    def action(self, items):
        return f'Действие: {items[0]}'

    def cycle(self, items):
        cycle_type = 'раз' if isinstance(items[1], str) else 'пока'
        return f'Цикл {cycle_type}: {items[2]}'

    def if_expression(self, items):
        return f'Если {items[0]}, то:\n{"\n".join(items[2:-1])}'


In [5]:
input_string = """
использовать Робот 
алг ура
нач
нц пока сверху свободно 
  если не справа свободно то
    закрасить
  все
кц
закрасить
нц 3 раз
влево
кц
кон
"""

try:
    # Парсинг входной строки и вывод результата
    parsed_tree = parser.parse(input_string)
    
    # Отладочный вывод структуры дерева разбора
    print("Parsed Tree:", parsed_tree.pretty())
    
    transformed_result = KumirTransformer().transform(parsed_tree)
    
    print(transformed_result)
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Parsed Tree: start
  alg
    name	 ура
    body
      command
        cycle
          пока
          condition
            direction	сверху
            wall	свободно
          command
            if_expression
              condition
                не
                direction	справа
                wall	свободно
              command
                action	закрасить
      command
        action	закрасить
      command
        cycle
          3
          раз
          command
            action	влево

Ошибка при разборе ввода: Error trying to process rule "start":

list index out of range
